<h1><center><b>Computer Laboratory III : Computational Intelligence Assignment - 2</b></center></h1>


## ------------------------------------------------------------------------------------------------------------

### **Problem Statement:**
Optimization of genetic algorithm parameter in hybrid genetic algorithm-neural network
modelling: Application to spray drying of coconut milk.


## ------------------------------------------------------------------------------------------------------------

#### **Importing Libraries**

In [1]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from deap import base, creator, tools, algorithms



## ------------------------------------------------------------------------------------------------------------

#### **Step 1: Defining Neural Network Model**

In [9]:
def create_nn_model(X_train, y_train):
    model = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    return model

## ------------------------------------------------------------------------------------------------------------

#### **Step 2: Defining Fitness Function for GA**

In [4]:
def evaluate(individual):
    inlet_temp, feed_flow, atomization_pressure = individual
    predicted_output = nn_model.predict([[inlet_temp, feed_flow, atomization_pressure]])
    fitness = predicted_output[0]
    return fitness,



## ------------------------------------------------------------------------------------------------------------

#### **Step 3: Setting up GA**

In [5]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 100, 200)  # Inlet temperature range
toolbox.register("attr_flow", np.random.uniform, 10, 50)     # Feed flow rate range
toolbox.register("attr_pressure", np.random.uniform, 1, 5)   # Atomization pressure range
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_float, toolbox.attr_flow, toolbox.attr_pressure), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)



## ------------------------------------------------------------------------------------------------------------

#### **Step 4: Training Sample Data**

In [6]:
X_train = np.array([[150, 30, 3], [160, 35, 4], [170, 40, 2]]) 
y_train = np.array([0.85, 0.90, 0.80]) 
nn_model = create_nn_model(X_train, y_train)



## ------------------------------------------------------------------------------------------------------------

#### **Step 5: Running Genetic Algorithm**

In [7]:
population = toolbox.population(n=50)
ngen, cxpb, mutpb = 40, 0.7, 0.2
algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=True)

gen	nevals
0  	50    
1  	35    
2  	41    
3  	39    
4  	41    
5  	34    
6  	41    
7  	36    
8  	41    
9  	39    
10 	38    
11 	30    
12 	40    
13 	43    
14 	41    
15 	34    
16 	43    
17 	39    
18 	37    
19 	40    
20 	42    
21 	42    
22 	36    
23 	43    
24 	39    
25 	36    
26 	35    
27 	39    
28 	41    
29 	36    
30 	38    
31 	37    
32 	35    
33 	37    
34 	38    
35 	39    
36 	37    
37 	39    
38 	30    
39 	38    
40 	38    


([[259.17549970259097, -10.723881664773943, -40.85642517996574],
  [258.8520779287888, -5.872078119420235, -40.7823396309528],
  [254.3166302771292, -10.356391960152381, -39.811588673277775],
  [259.40769638841635, -9.223066650303375, -38.531598597649584],
  [259.5796791643992, -9.770617237377365, -40.86158930210097],
  [259.5796791643992, -9.770617237377365, -40.86158930210097],
  [261.42712178559754, -10.148043122829812, -38.89267878249608],
  [260.12197370025893, -10.049343845333976, -39.858855582957105],
  [250.5282779265452, -10.158892906404489, -39.27830117175625],
  [255.88038865162235, -10.759271057537521, -42.036823463404104],
  [258.36266193290794, -9.469851151860913, -38.80248784831768],
  [260.12197370025893, -10.049343845333976, -39.858855582957105],
  [256.23733999560835, -10.754381449322151, -39.7273738850002],
  [248.49378262870388, -10.96746352752842, -39.43834890502085],
  [253.65371089585076, -10.630598181738476, -40.49825656949528],
  [254.90810599066344, -10.302701

## ------------------------------------------------------------------------------------------------------------

#### **Step 6: Extracting Best Solution**

In [10]:
best_individual = tools.selBest(population, k=1)[0]
print("Best Parameters:", best_individual)
print("Predicted Powder Yield:", evaluate(best_individual)[0])

Best Parameters: [260.6644623035142, -10.373473423325152, -41.25341521683167]
Predicted Powder Yield: 61.54318532759542


## ------------------------------------------------------------------------------------------------------------